# CCMM - Data Format

In [12]:
# LOADING JSON DATA
import json

file_path = "D:\\Python\\python-works\\ccmm-companies-data-format\\ccmm-company-details.json"
with open(file_path,"r", encoding='utf-8') as file:
    data = json.load(file)

print(len(data))

2465


There are `2465` companies data.
```js
// SAMPLE DATA JSON
{
    name: "",
    link: "",
    SCIAN: "",
    address_info: [],
    main_delegate: [
        {
            name: "",
            url: "",
            position: "",
            social_medias: [""]
        }
    ],
    secondary_delegate: [
        {
            name: "",
            url: "",
            position: "",
            social_medias: [""]
        }
    ]
}

```

In [4]:
# FUNCTION TO RECOGNIZE WEBSITE, ADDRESS, TELEPHONE
# FROM THE address_info LIST
import re

def extract_address_info(address_info):
    result = {
        "Address": "",
        "Telephone": "",
        "Website": ""
    }
    
    for item in address_info:
        if item.startswith('tel:'):
            result['Telephone'] = item.replace('tel:', '')
        elif item.startswith('http://') or item.startswith('https://'):
            result['Website'] = item
        else:
            result['Address'] = item
    
    return result


{'Address': '74 Avenue du Dauphiné Candiac', 'Telephone': '+14504662275', 'Website': ''}


In [19]:
# THIS CODE ONLY TO SEPARATE THE MULTIPLE
# SCIAN's WHICH ARE COMBINED IN SCIAN KEY

scian_set = set()

for com in data:
    if "SCIAN" in com:
        scian_set.add(len(com["SCIAN"]))
    else:
        print(com)
        break;

print(scian_set)


{0, 1, 2, 3}


In [16]:
# CODE TO SPLIT THE SCIAN CODES
import re

def split_scian_codes(scian_string):
    scian_codes = re.split(r'(?<!\d)(\d+) - ', scian_string)
    
    if scian_codes[0] == '':
        scian_codes.pop(0)
    
    scian_codes = [f'{code} - {desc}' for code, desc in zip(scian_codes[0::2], scian_codes[1::2])]
    
    return scian_codes


In [17]:
# THIS CODE ONLY TO COUNT THE MULTIPLE
# SCIAN's WHICH ARE COMBINED IN SCIAN KEY

scian_set = set()

for com in data:
    if "SCIAN" in com :
        scian_set.add(len(split_scian_codes(com["SCIAN"])))
    else:
        print(com)
        break;

print(scian_set)

{1, 2, 3}


In [18]:
# SPLITTING & VERIFYING MODIFICAITON
for com in data:
    if "SCIAN" in com and len(com["SCIAN"]) > 0:
        com["SCIAN"] = split_scian_codes(com["SCIAN"][0])


{'name': 'Accenture', 'link': 'https://www.ccmm.ca/fr/entreprises/accenture/', 'SCIAN': ['54161 - Services de conseils en gestion'], 'address_info': ['5 Place Ville-Marie suite 1520 Montréal', 'tel:+15148474027', 'http://www.accenture.com'], 'main_delegate': [{'name': 'Martine Lapointe', 'url': '#/', 'position': 'Directrice du Bureau de Montréal', 'social_medias': ['mailto:martine.lapointe@accenture.com']}], 'secondary_delegate': [{'name': 'Grace Ayoub', 'url': '/fr/membres/grace-ayoub/', 'position': 'Managing Director', 'social_medias': ['mailto:grace.ayoub@accenture.com']}, {'name': 'Kallia Mansour', 'url': '#/', 'position': 'Integrated Marketing Manager', 'social_medias': ['mailto:kallia.mansour@accenture.com']}, {'name': 'Souad Miloudi', 'url': '/fr/membres/souad-miloudi/', 'position': 'Assistante à la directoin', 'social_medias': ['mailto:souad.miloudi@accenture.com']}, {'name': 'Hadi Skalli', 'url': '/fr/membres/hadi-skalli/', 'position': 'Digital Business Intergrational Senior M

In [24]:
# SEPARATE COMPANY DETAILS
company_data = []

for com  in data:
    
    address_info_results = extract_address_info(com["address_info"])

    company_data.append({
        "Name": com["name"],
        "Webpage": com["link"],
        "Address": address_info_results["Address"],
        "Telephone": address_info_results["Telephone"],
        "Website": address_info_results["Website"],
        "SCIAN Code 1": com["SCIAN"][0] if len(com["SCIAN"]) > 0 else '',
        "SCIAN Code 2": com["SCIAN"][1] if len(com["SCIAN"]) > 1 else '',
        "SCIAN Code 3": com["SCIAN"][2] if len(com["SCIAN"]) > 2 else ''
    })


In [27]:
# CONVERT JSON LIST TO DATAFRAME
# ADD SAVE INTO EXCEL
import pandas as pd

companies_df = pd.json_normalize(company_data)
companies_df.to_excel("company_details.xlsx",sheet_name="Company Detail",)

print(companies_df.head())

                                             Name  \
0                                       Accenture   
1                       ALLIED PROPERTIES MGT LTD   
2                    Alstom Transport Canada Inc.   
3  Association de la construction du Québec - ACQ   
4                 Autorité des marchés financiers   

                                             Webpage  \
0      https://www.ccmm.ca/fr/entreprises/accenture/   
1  https://www.ccmm.ca/fr/entreprises/allied-prop...   
2  https://www.ccmm.ca/fr/entreprises/alstom-tran...   
3  https://www.ccmm.ca/fr/entreprises/association...   
4  https://www.ccmm.ca/fr/entreprises/autorite-de...   

                                             Address     Telephone  \
0            5 Place Ville-Marie suite 1520 Montréal  +15148474027   
1  7000-111 Boul Robert-Bourassa Bureau 100 Montréal  +15148689002   
2             1101, rue Parent 18e étage Saint-Bruno  +15147641725   
3             9200, boul. Métropolitain Est Montréal  +151435406